In [1]:
# install MeCab
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
# install NEologd
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a
# install neologdn
!pip install neologdn
# install emoji
!pip install emoji

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [0]:
import numpy as np
import pandas as pd
import MeCab
import re
import neologdn
import string
import emoji
import torch
import torchtext
from torchtext.vocab import Vectors

In [3]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/RionTweetClassifier/data/rion_tweet.csv')
df = df.drop(columns='created_at')
df.head()

,text,label
0,@katsudion かつでぃおんさんとまたLOLしたいです！,1
1,@vtubersyuki 久々LOLコラボ・・・うおお,1
2,@water_goodtaste コラボじゃから(..¯﹃¯ ..),1
3,明日（今日）\n\n起きて元気だったら朝活\nお昼元気だったらあつ森\n夜23:00頃LOL...,1
4,鰻のメンバーの人コミュニティ見ててね,1


In [0]:
# 前処理
def preprocessing_text(text):
    # 英語の小文字化(表記揺れの抑制)
    text = text.lower()
    # URLの除去(neologdnの後にやるとうまくいかないかも(URL直後に文章が続くとそれも除去される)))
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', '', text)
    # tweetの前処理
    text = re.sub(r"@([A-Za-z0-9_]+) ", '', text) # リプライ
    text = re.sub(r'#(\w+)', '', text) # ハッシュタグ
    # neologdnを用いて文字表現の正規化(全角・半角の統一と重ね表現の除去)
    text = neologdn.normalize(text)
    # 数字を全て0に置換(解析タスク上、数字を重要視しない場合は語彙数増加を抑制するために任意の数字に統一したり除去することもある)
    text = re.sub(r'[0-9０-９]+', '0', text)
    # 半角記号の除去
    text = re.sub(r'[!-/:-@【】[-`{-~]', "", text)
    # 改行
    text = re.sub('\n', '', text)
    # 絵文字
    text = ''.join(['' if c in emoji.UNICODE_EMOJI else c for c in text])
    # 中黒や三点リーダ
    text = re.sub(r'[・…]', '', text)
    return text

def tokenizer_mecab(text, preprocessing=False):
    # テキスト前処理
    if preprocessing:
        text = preprocessing_text(text)
    # 分かち書き
    tagger = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        word = node.surface
        word_list.append(word)
        node = node.next
    return ' '.join(word_list)

def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_mecab(text)
    
    return ret

In [7]:
df_new = df.copy()
df_new['tokenized'] = df_new['text'].apply(lambda x : tokenizer_with_preprocessing(x))
df_new.head()

,text,label,tokenized
0,@katsudion かつでぃおんさんとまたLOLしたいです！,1,かつ で ぃおんさんとまた lol し たい です
1,@vtubersyuki 久々LOLコラボ・・・うおお,1,久々 lol コラボ うおお
2,@water_goodtaste コラボじゃから(..¯﹃¯ ..),1,コラボ じゃ から ¯﹃¯
3,明日（今日）\n\n起きて元気だったら朝活\nお昼元気だったらあつ森\n夜23:00頃LOL...,1,明日 今日 起き て 元気 だっ たら 朝活 お昼 元気 だっ たら あつ 森 夜 00 ...
4,鰻のメンバーの人コミュニティ見ててね,1,鰻 の メンバー の 人 コミュニティ 見 て て ね


In [8]:
df_new[df_new.tokenized == ' ']

,text,label,tokenized
23,@VTM_himari 😨,0,
39,#新しいプロフィール画像 https://t.co/sDCrdyzM6v,1,
58,https://t.co/8HWuA97EdO,1,
76,https://t.co/Hltgwt5yIL https://t.co/6YjbNs2Fd5,1,
118,@Hakase_Fuyuki 😭😭😭😭🙏🙏🙏🙏,1,
288,@shysnDYo 😨,1,
370,@nyangagan ；；,1,
423,#V女子雪山,1,
427,https://t.co/UVo7ywwvYW,1,
428,#V女子雪山,1,


In [0]:
corpus = df.drop(index=df_new[df_new.tokenized == ' '].index)
corpus.to_csv('/content/drive/My Drive/Colab Notebooks/NLP/RionTweetClassifier/data/rion_corpus.csv', index=False, header=True)